<h3 style="color:red;">Make sure the Docker service running before running this notebook</h3>

# Section 1: Launching HaGrid nodes

In [1]:
! hagrid launch {"Domain_1"} to docker:8001 --build-src=0.7.0

✅ Updated HAGrid from branch: 0.7.0m
⠸ Updating HAGrid from branch: 0.7.0
⠼ Checking for Docker Service   ice   
✅ Docker service is running
✅ Git 2.37.1
✅ Docker 20.10.22
✅ Docker Compose 2.15.1


 _   _       _     _                 _   _                       _
| | | |     | |   | |               | | | |                     | |
| |_| | ___ | | __| |   ___  _ __   | |_| | __ _ _ __ _ __ _   _| |
|  _  |/ _ \| |/ _` |  / _ \| '_ \  |  _  |/ _` | '__| '__| | | | |
| | | | (_) | | (_| | | (_) | | | | | | | | (_| | |  | |  | |_| |_|
\_| |_/\___/|_|\__,_|  \___/|_| |_| \_| |_/\__,_|_|  |_|   \__, (_)
                                                            __/ |
                                                           |___/
        
Launching a PyGrid Domain node on port 8001!

  - NAME: domain_1
  - RELEASE: production
  - ARCH: linux/arm64
  - TYPE: domain
  - DOCKER_TAG: latest
  - HAGRID_VERSION: 0.2.126
  - PORT: 8001
  - DOCKER COMPOSE: v2.15.1


⠋ Launching Containers  
  ✅ Pu

In [2]:
from utils import *

domain_user = sy.login(email="info@openmined.org", 
                      password="changethis", 
                      port=8001)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into domain_1... done!


In [3]:
! docker compose ls

NAME                STATUS              CONFIG FILES
domain_1            running(11)         /Users/neeleshkarthikeyan/.hagrid/PySyft/packages/grid/docker-compose.yml
domain_2            running(11)         /Users/neeleshkarthikeyan/.hagrid/PySyft/packages/grid/docker-compose.yml
local_network       running(11)         /Users/neeleshkarthikeyan/.hagrid/PySyft/packages/grid/docker-compose.yml


In [4]:
! docker container ls

CONTAINER ID   IMAGE                                 COMMAND                  CREATED         STATUS         PORTS                                                                               NAMES
72f84f49c7f9   openmined/grid-frontend:latest        "/docker-entrypoint.…"   4 minutes ago   Up 4 minutes   80/tcp                                                                              domain_1-frontend-1
367b5187f6f4   openmined/grid-frontend:latest        "/docker-entrypoint.…"   4 minutes ago   Up 4 minutes   80/tcp                                                                              domain_2-frontend-1
13319163f6e7   chrislusf/seaweedfs:latest            "/entrypoint.sh serv…"   4 minutes ago   Up 4 minutes   7333/tcp, 8080/tcp, 8333/tcp, 8888/tcp, 9333/tcp, 18080/tcp, 18888/tcp, 19333/tcp   domain_2-seaweedfs-1
f9ab9a156c0e   openmined/grid-backend:latest         "/start.sh"              4 minutes ago   Up 4 minutes                                                       

In [5]:
domain_user.apply_to_network(sy.login("localhost", 8000))

Connecting to localhost... done! 	 Logging into local_network... as GUEST...done!
[1/4] Checking Syft Versions                            
[2/4] Joining Network                     o Secure VPN .[2/4] Joining Network ...
[3/4] Connecting to Secure VPN                              
ConnectionTimeoutError: Node was not able to process your request in time.  Retrying 3  ...
[4/4] Registering on the Secure VPN                                   


---

# Section 2 : Train the dataset and Upload Weights

In [6]:
num_clients = 2
manual_seed = 101
trainloaders, valloaders, testloaders = load_datasets(num_clients, manual_seed)

print("\nNumber of train images in domain 2:", len(trainloaders[0]))
print("\nNumber of val images in domain 2:", len(valloaders[0]))
print("\nNumber of test images in domain 2:", len(testloaders[0]))

initial_model = Net()
optimizer = optim.SGD(initial_model.parameters(), lr=0.001, momentum=0.9)

i, j = 0, 1
for param in initial_model.parameters():
    if i % 2 == 0:
        param.data = torch.FloatTensor(domain_user.datasets[i]["Weight {}".format(j)].get().child.value)
    else:    
        param.data = torch.FloatTensor(domain_user.datasets[i]["Bias {}".format(j)].get().child.value)
        j += 1        
    i += 1
    

for epoch in range(1, 11):
    train(initial_model, trainloaders[0], epoch)
    loss, accuracy = test(initial_model, valloaders[0])
    
    loss = round(loss, 2)
    accuracy = round(accuracy, 2)
    
    print(f"Epoch {epoch}: Validation Loss: {loss}, Accuracy: {accuracy*100}%")

Files already downloaded and verified
Files already downloaded and verified

Number of train images in domain 2: 704

Number of val images in domain 2: 79

Number of test images in domain 2: 157


kj/filesystem-disk-unix.c++:1690: warning: PWD environment variable doesn't match current directory; pwd = /Users/neeleshkarthikeyan


Epoch 1: Validation Loss: 0.05, Accuracy: 49.0%
Epoch 2: Validation Loss: 0.04, Accuracy: 55.00000000000001%
Epoch 3: Validation Loss: 0.04, Accuracy: 55.00000000000001%
Epoch 4: Validation Loss: 0.05, Accuracy: 56.00000000000001%
Epoch 5: Validation Loss: 0.06, Accuracy: 54.0%
Epoch 6: Validation Loss: 0.08, Accuracy: 53.0%
Epoch 7: Validation Loss: 0.11, Accuracy: 52.0%
Epoch 8: Validation Loss: 0.13, Accuracy: 51.0%
Epoch 9: Validation Loss: 0.15, Accuracy: 51.0%
Epoch 10: Validation Loss: 0.18, Accuracy: 52.0%


In [8]:
i, j = 0, 0
for layer in initial_model.children():
        
    if "weight" in layer.state_dict():
        i += 1
            
        model_weights = layer.state_dict()["weight"]
        weights_tensor = sy.Tensor(model_weights).annotate_with_dp_metadata(lower_bound=-1, upper_bound=1, data_subject="PyTorch Weights")

        domain_user.load_dataset(assets={"Weight {}".format(i): weights_tensor},
                              name="Domain Weight {}".format(i),
                              description="Updated PyTorch weights sent by Domain.")

    if "bias" in layer.state_dict():
        j += 1

        model_bias = layer.state_dict()["bias"]
        bias_tensor = sy.Tensor(model_bias).annotate_with_dp_metadata(lower_bound=-1, upper_bound=1, data_subject="PyTorch Bias")

        domain_user.load_dataset(assets={"Bias {}".format(j): bias_tensor},
                              name="Domain Bias {}".format(j),
                              description="Updated PyTorch biases sent by Domain.")
        

Converting PyTorch tensor to numpy tensor for internal representation...
Tensor annotated with DP Metadata!
You can upload this Tensor to a domain node by calling `<domain_client>.load_dataset` and passing in this tensor as an asset.
Loading dataset... uploading...🚀                        

Uploading `Weight 1`: 100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 84.39it/s]

Dataset is uploaded successfully !!! 🎉

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!
Converting PyTorch tensor to numpy tensor for internal representation...
Tensor annotated with DP Metadata!
You can upload this Tensor to a domain node by calling `<domain_client>.load_dataset` and passing in this tensor as an asset.
Loading dataset... checking asset types...                              

Loading dataset... uploading...🚀                        

Uploading `Bias 1`: 100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 139.68it/s]


...

Dataset is uploaded successfully !!! 🎉

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!
Converting PyTorch tensor to numpy tensor for internal representation...
Tensor annotated with DP Metadata!
You can upload this Tensor to a domain node by calling `<domain_client>.load_dataset` and passing in this tensor as an asset.
Loading dataset... uploading...🚀                        

Uploading `Weight 5`: 100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 194.14it/s]


Dataset is uploaded successfully !!! 🎉

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!
Converting PyTorch tensor to numpy tensor for internal representation...
Tensor annotated with DP Metadata!
You can upload this Tensor to a domain node by calling `<domain_client>.load_dataset` and passing in this tensor as an asset.
Loading dataset... uploading...🚀                                                                                                                                             

Uploading `Bias 5`: 100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 216.40it/s]


Dataset is uploaded successfully !!! 🎉

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!


In [9]:
domain_user.datasets

Idx,Name,Description,Assets,Id
[0],Initial Weight 1,Initial PyTorch weights sent by Network.,"[""Weight 1""] -> Tensor",6a7d9a50-768d-4b50-91c3-7afc378c3d44
[1],Initial Bias 1,Initial PyTorch biases sent by Network.,"[""Bias 1""] -> Tensor",863b2e56-f7ac-4c65-a6f6-cc1773d5df7f
[2],Initial Weight 2,Initial PyTorch weights sent by Network.,"[""Weight 2""] -> Tensor",2d8b86bf-6136-45be-a55e-b32f745b4d75
[3],Initial Bias 2,Initial PyTorch biases sent by Network.,"[""Bias 2""] -> Tensor",165b16f5-7d38-4e06-abac-ff8ac19538bb
[4],Initial Weight 3,Initial PyTorch weights sent by Network.,"[""Weight 3""] -> Tensor",73ba01b9-b2d7-4cfd-a2b2-78018420fcfa
[5],Initial Bias 3,Initial PyTorch biases sent by Network.,"[""Bias 3""] -> Tensor",712206ef-e926-4e5a-945a-2dfa0221bb9f
[6],Initial Weight 4,Initial PyTorch weights sent by Network.,"[""Weight 4""] -> Tensor",56bb309d-80b1-4197-ace7-d8ef1b123f6a
[7],Initial Bias 4,Initial PyTorch biases sent by Network.,"[""Bias 4""] -> Tensor",c553b5cd-6f5a-4a97-8f75-11a7df37e413
[8],Initial Weight 5,Initial PyTorch weights sent by Network.,"[""Weight 5""] -> Tensor",aebe0022-575e-4711-b5c6-40ce78072970
[9],Initial Bias 5,Initial PyTorch biases sent by Network.,"[""Bias 5""] -> Tensor",4be73095-2d88-4cee-851a-6e944927d2ed


---

# Section 3: Test on Aggregated Weights

<h3 style="color:red;">Run the below cells after running Section 3 in Network notebook</h3>

In [10]:
domain_user.datasets

Idx,Name,Description,Assets,Id
[0],Initial Weight 1,Initial PyTorch weights sent by Network.,"[""Weight 1""] -> Tensor",6a7d9a50-768d-4b50-91c3-7afc378c3d44
[1],Initial Bias 1,Initial PyTorch biases sent by Network.,"[""Bias 1""] -> Tensor",863b2e56-f7ac-4c65-a6f6-cc1773d5df7f
[2],Initial Weight 2,Initial PyTorch weights sent by Network.,"[""Weight 2""] -> Tensor",2d8b86bf-6136-45be-a55e-b32f745b4d75
[3],Initial Bias 2,Initial PyTorch biases sent by Network.,"[""Bias 2""] -> Tensor",165b16f5-7d38-4e06-abac-ff8ac19538bb
[4],Initial Weight 3,Initial PyTorch weights sent by Network.,"[""Weight 3""] -> Tensor",73ba01b9-b2d7-4cfd-a2b2-78018420fcfa
[5],Initial Bias 3,Initial PyTorch biases sent by Network.,"[""Bias 3""] -> Tensor",712206ef-e926-4e5a-945a-2dfa0221bb9f
[6],Initial Weight 4,Initial PyTorch weights sent by Network.,"[""Weight 4""] -> Tensor",56bb309d-80b1-4197-ace7-d8ef1b123f6a
[7],Initial Bias 4,Initial PyTorch biases sent by Network.,"[""Bias 4""] -> Tensor",c553b5cd-6f5a-4a97-8f75-11a7df37e413
[8],Initial Weight 5,Initial PyTorch weights sent by Network.,"[""Weight 5""] -> Tensor",aebe0022-575e-4711-b5c6-40ce78072970
[9],Initial Bias 5,Initial PyTorch biases sent by Network.,"[""Bias 5""] -> Tensor",4be73095-2d88-4cee-851a-6e944927d2ed


In [11]:
trainloaders, valloaders, testloaders = load_datasets(num_clients, manual_seed)

loss, accuracy = test(initial_model, testloaders[0])
print(f"Initial model performance: \n\tLoss {round(loss, 2)}\n\tAccuracy: {round(accuracy*100, 2)}%")

agg_model = Net() # 47%

i, j = 20, 1
for param in agg_model.parameters():
    
    if i % 2 == 0:
        param.data = torch.FloatTensor(domain_user.datasets[i]["Weight {}".format(j)].get().child.value)
    
    else:    
        param.data = torch.FloatTensor(domain_user.datasets[i]["Bias {}".format(j)].get().child.value)
        j += 1        
        
    i += 1
    
    
loss, accuracy = test(agg_model, testloaders[0])
print(f"\nAggregated model performance:\n \tLoss {round(loss, 2)}\n\tAccuracy: {round(accuracy*100, 2)}%")

Files already downloaded and verified
Files already downloaded and verified
Initial model performance: 
	Loss 0.19
	Accuracy: 50.9%

Aggregated model performance:
 	Loss 0.13
	Accuracy: 53.94%


---